In [1]:
import os
import instructor
from openai import OpenAI
from dotenv import load_dotenv
from src.services.core.messages_service import MessagesService

load_dotenv()
client = instructor.patch(OpenAI(api_key=os.getenv("OPENAI_API_KEY")))

In [2]:
# Initialize vars
messages_service = MessagesService()
openai_model = "gpt-3.5-turbo"
user_interest_enum = None

# Initial system message
system_prompt = """You are an assistant that helps users find sushi restaurants and parking spots in Munich. 
When a user expresses interest in either sushi, parking, or both:
1. Use the set_user_interest function to record their preference
2. Provide relevant parking or restaurant information
3. ALWAYS respond with a natural language message summarizing the information for the user
"""

messages_service.add_system_message(system_prompt)

user_prompt = "I want to park my car"
user_prompt = "I want to eat sushi"
# user_prompt = "I want to park my car and eat sushi"
# user_prompt = input("eg. I am looking for a Sushi Restaurant in Munich.")
user_prompt = "Hey"

messages_service.add_user_message(user_prompt)
messages_service.print_messages()

{'role': 'system', 'content': 'You are an assistant that helps users find sushi restaurants and parking spots in Munich. \nWhen a user expresses interest in either sushi, parking, or both:\n1. Use the set_user_interest function to record their preference\n2. Provide relevant parking or restaurant information\n3. ALWAYS respond with a natural language message summarizing the information for the user\n'}
{'role': 'user', 'content': 'Hey'}


In [3]:
from src.models.intent.user_interest_intent import UserInterestIntent

# Get user interest
intent : UserInterestIntent = client.chat.completions.create(
    model=openai_model,
    messages=messages_service.get_messages(),
    response_model=UserInterestIntent,
)

messages_service.add_assistant_message(intent.assistant_message)
messages_service.add_function_message(UserInterestIntent.__name__, intent.interest)

user_interest_enum = intent.interest

messages_service.print_messages()

{'role': 'system', 'content': 'You are an assistant that helps users find sushi restaurants and parking spots in Munich. \nWhen a user expresses interest in either sushi, parking, or both:\n1. Use the set_user_interest function to record their preference\n2. Provide relevant parking or restaurant information\n3. ALWAYS respond with a natural language message summarizing the information for the user\n'}
{'role': 'user', 'content': 'Hey'}
{'role': 'assistant', 'content': 'Hello! How can I assist you today? Are you looking for sushi restaurants, parking spots, or both in Munich?'}
{'role': 'function', 'name': 'UserInterestIntent', 'content': <UserInterestEnum.ASSISTANT: 'ASSISTANT'>}


In [4]:
# user_prompt = "List all parking spots"
user_prompt = "How can i pay for parking?"
user_prompt = "Where can i eat german sushi?"
user_prompt = "Tell a joke"
messages_service.add_user_message(user_prompt)

### Choose response format based on user intend
Its either Parking or Sushi

In [5]:
from src.services.user.user_interest_service import get_user_interest

response_model, intent_service = get_user_interest(user_interest_enum)

intent = client.chat.completions.create(
    model=openai_model,
    messages=messages_service.get_messages(),
    response_model=response_model,
)

messages_service.add_assistant_message(intent.assistant_message)
messages_service.print_last_message()
messages_service.add_function_message(intent.__class__.__name__, intent)
messages_service.print_last_message()

if intent_service is not None:
    result = intent_service.handle_intent(intent)
    messages_service.add_function_message(intent_service.handle_intent.__name__, result)
    messages_service.print_last_message()

{'role': 'assistant', 'content': 'I believe a good joke about sushi might lift your spirits! Why did the sushi blush? Because it saw the seaweed!'}
{'role': 'function', 'name': 'AssistantModel', 'content': '{"assistant_message":"I believe a good joke about sushi might lift your spirits! Why did the sushi blush? Because it saw the seaweed!"}'}


In [6]:
from src.models.intent.assistant_model import AssistantModel

response : AssistantModel = client.chat.completions.create(
    model=openai_model,
    messages=messages_service.get_messages(),
    response_model=AssistantModel,
)

messages_service.add_assistant_message(response.assistant_message)
messages_service.print_last_message()

{'role': 'assistant', 'content': 'I believe a good joke about sushi might lift your spirits! Why did the sushi blush? Because it saw the seaweed!'}


### Use it like a chat

In [7]:
from src.models.intent.user_interest_intent import UserInterestIntent

for i in range(2):
    
    # get user input
    user_prompt = input("User: ")
    messages_service.add_user_message(user_prompt)
    messages_service.print_last_message()


    # Get user interest each time to shrink down the response models
    intent : UserInterestIntent = client.chat.completions.create(
        model=openai_model,
        messages=messages_service.get_messages(),
        response_model=UserInterestIntent,
    )

    messages_service.add_assistant_message(intent.assistant_message)
    messages_service.print_last_message()
    messages_service.add_function_message(UserInterestIntent.__name__, intent.interest)
    messages_service.print_last_message()

    user_interest_enum = intent.interest
    response_model, intent_service = get_user_interest(user_interest_enum)

    # get user intent
    intent = client.chat.completions.create(
        model=openai_model,
        messages=messages_service.get_messages(),
        response_model=response_model,
    )

    messages_service.add_function_message(intent.__class__.__name__, intent)
    messages_service.print_last_message()

    
    # handle intent
    if intent_service is not None:
        result = intent_service.handle_intent(intent)
        messages_service.add_function_message(intent_service.handle_intent.__name__, result)
        messages_service.print_last_message()

        # Assistant help to summarize the data
        response = client.chat.completions.create(
            model=openai_model,
            messages=messages_service.get_messages(),
            response_model=AssistantModel,
        )

        messages_service.add_assistant_message(response.assistant_message)
        messages_service.print_last_message()


{'role': 'user', 'content': ''}
{'role': 'assistant', 'content': 'I believe a good joke about sushi might lift your spirits! Why did the sushi blush? Because it saw the seaweed!'}
{'role': 'function', 'name': 'UserInterestIntent', 'content': <UserInterestEnum.ASSISTANT: 'ASSISTANT'>}
{'role': 'function', 'name': 'AssistantModel', 'content': '{"assistant_message":"It seems like you might be interested in sushi or parking. How can I assist you today?"}'}


In [8]:
# reset jupiter kernel
# %reset -f